# Trip Mapper!

(Developed by Benjamin in 2023, prior to Capstone)

Skip to "Working Version" for 2025 update

In [ ]:
### Old implementation before optimizations: 


## Getting coords and distances

"""
def get_coordinates_and_distances(cities):
    coordinates = {}
    distances = {}
    
    # Get Coordinates using Geocoding API
    for city in cities:
        geocode_result = gmaps.geocode(city)
        lat = geocode_result[0]['geometry']['location']['lat']
        lng = geocode_result[0]['geometry']['location']['lng']
        coordinates[city] = (lat, lng)
        
    # Get Distances using Distance Matrix API
    for i, origin in enumerate(cities):
        for j, destination in enumerate(cities):
            if i < j:  # Avoid duplicate calculations
                distance_result = gmaps.distance_matrix(origin, destination)
                try:
                    distance = distance_result['rows'][0]['elements'][0]['distance']['value']  # Get distance in meters
                    distances[(origin, destination)] = distance
                    distances[(destination, origin)] = distance  # Distance is the same in both directions
                except KeyError:
                    print(f"Error: Unable to get the distance between {origin} and {destination}.")
                    print("Please make sure the city names are correct and there is a route between them.")
                    return None, None
    
    return coordinates, distances


def tsp_bruteforce(coordinates, distances):
    cities = list(coordinates.keys())
    n = len(cities)
    if n > 10:
        raise ValueError("Too many cities, computation will take too long.")
    all_paths = permutations(range(n))
    min_path = None
    min_distance = maxsize
    
    for path in all_paths:
        distance = 0
        for i in range(n - 1):
            distance += distances[(cities[path[i]], cities[path[i + 1]])]
        if distance < min_distance:
            min_distance = distance
            min_path = path
            
    return [cities[i] for i in min_path]


def draw_map(m, coordinates, path):
    m.shadedrelief()  # Adding shaded relief for more colorful map
    m.drawcoastlines()
    m.drawcountries()
    m.drawstates()
    
    # Draw the cities
    for city, coord in coordinates.items():
        lat, lon = coord
        xpt, ypt = m(lon, lat)
        m.plot(xpt, ypt, 'ro')  # Changed to red dot
    
    # Draw the path with arrows
    for i in range(len(path) - 1):
        start_city, end_city = coordinates[path[i]], coordinates[path[i + 1]]
        m.drawgreatcircle(start_city[1], start_city[0], end_city[1], end_city[0], linewidth=2, color='red')
        # You can further implement arrow drawing here

if __name__ == "__main__":
    user_input = input("Enter a comma-separated list of cities: ")
    selected_cities = [city.strip() for city in user_input.split(",")]

    # Prepend and append 'Knoxville' to the list
    selected_cities = ['Knoxville'] + selected_cities + ['Knoxville']
    
    coordinates, distances = get_coordinates_and_distances(selected_cities)
if coordinates is None or distances is None:
    print("Exiting due to errors.")
else:
    optimal_path = tsp_bruteforce(coordinates, distances)
    m = Basemap(
        projection='merc',
        llcrnrlat=-60,
        urcrnrlat=80,
        llcrnrlon=-180,
        urcrnrlon=180,
        lat_ts=20,
        resolution='i'
    )
    plt.figure(figsize=(10, 10))
    draw_map(m, coordinates, optimal_path)
    plt.show()
"""

## Haversine formula to calculate the distance between two points on the Earth's surface

"""
def haversine(lat1, lon1, lat2, lon2):
    R = 6371000  # Radius of Earth in meters
    d_lat = radians(lat2 - lat1)
    d_lon = radians(lon2 - lon1)
    a = (sin(d_lat / 2) * sin(d_lat / 2) +
         cos(radians(lat1)) * cos(radians(lat2)) * 
         sin(d_lon / 2) * sin(d_lon / 2))
    c = 2 * atan2(sqrt(a), sqrt(1 - a))
    distance = R * c
    return distance  # Distance in meters

def get_coordinates_and_distances(cities):
    coordinates = {}
    distances = {}
    
    # Get Coordinates using Geocoding API
    for city in cities:
        geocode_result = gmaps.geocode(city)
        lat = geocode_result[0]['geometry']['location']['lat']
        lng = geocode_result[0]['geometry']['location']['lng']
        coordinates[city] = (lat, lng)
        
    # Calculate Distances using Haversine Formula
    for origin in cities:
        for destination in cities:
            if origin != destination:
                distance = haversine(coordinates[origin][0], coordinates[origin][1], coordinates[destination][0], coordinates[destination][1])
                distances[(origin, destination)] = distance
    
    return coordinates, distances

def tsp_bruteforce(coordinates, distances):
    cities = list(coordinates.keys())
    n = len(cities)
    if n > 10:
        raise ValueError("Too many cities, computation will take too long.")
    all_paths = permutations(range(n))  # Generate permutations of indices, not city names
    min_path = None
    min_distance = maxsize
    
    for path in all_paths:
        distance = sum(distances[(cities[path[i]], cities[path[i + 1]])] for i in range(n - 1))
        if distance < min_distance:
            min_distance = distance
            min_path = path
            
    return [cities[i] for i in min_path]

def insert_knoxville(coordinates, distances, optimal_path):
    knoxville_coords = get_coordinates_and_distances(['Knoxville'])[0]['Knoxville']
    coordinates['Knoxville'] = knoxville_coords  # add Knoxville's coordinates to the coordinates dict
    optimal_path = ['Knoxville'] + optimal_path + ['Knoxville']  # append and prepend Knoxville to the optimal path list
    return optimal_path

def draw_map(m, coordinates, path):
    m.shadedrelief(scale=0.2)  # Natural and bright color
    m.drawcoastlines()
    m.drawcountries()  # Only national borders are shown
    
    # Draw the cities
    for city, coord in coordinates.items():
        lat, lon = coord
        xpt, ypt = m(lon, lat)
        m.scatter(xpt, ypt, c='black', edgecolors='white', s=87.5, linewidths=2, zorder=2)  # Increase size and border thickness
    
    # Draw the path
    for i in range(len(path) - 1):
        start_city, end_city = coordinates[path[i]], coordinates[path[i + 1]]
        m.drawgreatcircle(start_city[1], start_city[0], end_city[1], end_city[0], linewidth=3, linestyle='dashed', color='red')  # Dashed red line


if __name__ == "__main__":
    user_input = input("Enter a comma-separated list of cities: ")
    selected_cities = [city.strip() for city in user_input.split(",")]
    
    coordinates, distances = get_coordinates_and_distances(selected_cities)
    optimal_path = tsp_bruteforce(coordinates, distances)
    optimal_path_with_knoxville = insert_knoxville(coordinates, distances, optimal_path)
    
    plt.figure(figsize=(19.2, 9.6))  # Initialize figure size here
    
    m = Basemap(
        projection='moll',  # Mollweide projection
        lon_0=0,  # Central longitude
        resolution='l'  # low resolution for faster rendering, use 'i' or 'h' for higher resolution
    )
    
    m.drawmapboundary(fill_color='lightblue')  # Water is light blue
    m.fillcontinents(color='lightgreen', lake_color='lightblue')  # Land is light green
    m.drawparallels(np.arange(-90.,120.,30.))
    m.drawmeridians(np.arange(0.,420.,60.))
    
    draw_map(m, coordinates, optimal_path_with_knoxville)
    plt.show()
"""

## TSP NN, Two-Opt, Main:

"""
def tsp_nearest_neighbor(coordinates, distances):
    cities = list(coordinates.keys())
    unvisited = set(cities)
    path = [cities[0]]
    unvisited.remove(cities[0])
    current_city = cities[0]
    while unvisited:
        nearest_city = min(unvisited, key=lambda city: distances[(current_city, city)])
        path.append(nearest_city)
        current_city = nearest_city
        unvisited.remove(nearest_city)
    return path

def two_opt(path, distances):
    improvement = True
    while improvement:
        improvement = False
        for i in range(1, len(path) - 1):
            for j in range(i + 1, len(path)):
                if j - i == 1 or j == len(path) - 1: continue  # skip the adjacent indices and the last index
                old_dist = distances[(path[i], path[i - 1])] + distances[(path[j], path[j + 1])]
                new_dist = distances[(path[i], path[j])] + distances[(path[i - 1], path[j + 1])]
                if new_dist < old_dist:
                    path[i:j] = reversed(path[i:j])
                    improvement = True
                    break  # Break out of the inner loop as soon as an improvement is found
            if improvement: break  # Break out of the outer loop as well
    return path

if __name__ == "__main__":
    user_input = input("Enter a comma-separated list of cities: ")
    selected_cities = [city.strip() for city in user_input.split(",")]
    
    coordinates, distances = get_coordinates_and_distances(selected_cities)
    optimal_path = tsp_nearest_neighbor(coordinates, distances)
    optimal_path = two_opt(optimal_path, distances)
    optimal_path_with_knoxville = insert_knoxville(coordinates, distances, optimal_path)
    
    plt.figure(figsize=(19.2, 9.6))  # Initialize figure size here
    m = Basemap(projection='moll', lon_0=0, resolution='l')
    m.drawmapboundary(fill_color='lightblue')
    m.fillcontinents(color='lightgreen', lake_color='lightblue')
    m.drawparallels(np.arange(-90.,120.,30.))
    m.drawmeridians(np.arange(0.,420.,60.))
    draw_map(m, coordinates, optimal_path_with_knoxville)
    plt.show()
"""

In [ ]:
### Optimizations of old version (still using now-defunct library)

"""
def haversine(lat1, lon1, lat2, lon2):
    R = 6371000  # Radius of Earth in meters
    d_lat = radians(lat2 - lat1)
    d_lon = radians(lon2 - lon1)
    a = (sin(d_lat / 2) * sin(d_lat / 2) +
         cos(radians(lat1)) * cos(radians(lat2)) * 
         sin(d_lon / 2) * sin(d_lon / 2))
    c = 2 * atan2(sqrt(a), sqrt(1 - a))
    distance = R * c
    return distance  # Distance in meters

def get_coordinates_and_distances(cities):
    coordinates = {}
    distances = {}
    
    # Get Coordinates using Geocoding API
    for city in cities:
        geocode_result = gmaps.geocode(city)
        lat = geocode_result[0]['geometry']['location']['lat']
        lng = geocode_result[0]['geometry']['location']['lng']
        coordinates[city] = (lat, lng)
        
    # Calculate Distances using Haversine Formula
    for origin in cities:
        for destination in cities:
            if origin != destination and (origin, destination) not in distances and (destination, origin) not in distances:
                distance = haversine(coordinates[origin][0], coordinates[origin][1], coordinates[destination][0], coordinates[destination][1])
                distances[(origin, destination)] = distance
                distances[(destination, origin)] = distance  # Distance is the same in both directions
                
    return coordinates, distances
def tsp_nearest_neighbor(coordinates, distances):
    unvisited = list(coordinates.keys())
    start_city = unvisited.pop(0)  # Start from the first city
    path = [start_city]
    
    while unvisited:
        current_city = path[-1]
        next_city = min((city for city in unvisited), key=lambda city: distances[(current_city, city)])
        path.append(next_city)  # Corrected line
        unvisited.remove(next_city)
        
    path.append(path[0])  # Return to the starting city
    return path

def two_opt(path, distances):
    improvement = True
    while improvement:
        improvement = False
        for i in range(1, len(path) - 1):
            if improvement:
                break
            for j in range(i + 1, len(path)):
                if j - i == 1 or j == len(path) - 1: continue
                old_dist = distances[(path[i - 1], path[i])] + distances[(path[j], path[j + 1])]
                new_dist = distances[(path[i - 1], path[j])] + distances[(path[i], path[j + 1])]
                if new_dist < old_dist:
                    path[i:j + 1] = reversed(path[i:j + 1])
                    improvement = True
                    break
                    path.append(path[0])
    return path

def insert_knoxville(coordinates, distances, optimal_path):
    knoxville_coords = get_coordinates_and_distances(['Knoxville'])[0]['Knoxville']
    coordinates['Knoxville'] = knoxville_coords  # add Knoxville's coordinates to the coordinates dict
    optimal_path = ['Knoxville'] + optimal_path + ['Knoxville']  # append and prepend Knoxville to the optimal path list
    return optimal_path

def draw_map(m, coordinates, path):
    m.shadedrelief(scale=0.2)  # Natural and bright color
    m.drawcoastlines()
    m.drawcountries()  # Only national borders are shown
    
    # Draw the cities
    for city, coord in coordinates.items():
        lat, lon = coord
        xpt, ypt = m(lon, lat)
        m.scatter(xpt, ypt, c='black', edgecolors='white', s=87.5, linewidths=2, zorder=2)  # Increase size and border thickness
    
    # Draw the path
    for i in range(len(path) - 1):
        start_city, end_city = coordinates[path[i]], coordinates[path[i + 1]]
        m.drawgreatcircle(start_city[1], start_city[0], end_city[1], end_city[0], linewidth=3, linestyle='dashed', color='red')  # Dashed red line


if __name__ == "__main__":
    user_input = input("Enter a comma-separated list of cities: ")
    selected_cities = [city.strip() for city in user_input.split(",")]
    
    coordinates, distances = get_coordinates_and_distances(selected_cities)
    optimal_path = tsp_nearest_neighbor(coordinates, distances)
    optimal_path = two_opt(optimal_path, distances)
    optimal_path_with_knoxville = insert_knoxville(coordinates, distances, optimal_path)  # Define this before drawing the map
    
    plt.figure(figsize=(19.2, 9.6))  # Initialize figure size here
    
    m = Basemap(
        projection='moll',  # Mollweide projection
        lon_0=0,  # Central longitude
        resolution='l'  # low resolution for faster rendering, use 'i' or 'h' for higher resolution
    )
    
    m.drawmapboundary(fill_color='lightblue')  # Water is light blue
    m.fillcontinents(color='lightgreen', lake_color='lightblue')  # Land is light green
    m.drawparallels(np.arange(-90.,120.,30.))
    m.drawmeridians(np.arange(0.,420.,60.))
    
    draw_map(m, coordinates, optimal_path_with_knoxville)  # Correct the variable name here
    plt.show()"
    """

# Working Version (No depreciated libraries)

In [ ]:
import googlemaps
import matplotlib.pyplot as plt
import numpy as np

# Cartopy imports:
import cartopy.crs as ccrs
import cartopy.feature as cfeature
from geographiclib.geodesic import Geodesic

from itertools import permutations
from sys import maxsize
from math import radians, cos, sin, sqrt, atan2
import heapq

# API Key --TO REDACT
api_key = ""
gmaps = googlemaps.Client(key=api_key)

In [ ]:
def haversine(lat1, lon1, lat2, lon2):
    R = 6371000  # Radius of Earth in meters
    d_lat = radians(lat2 - lat1)
    d_lon = radians(lon2 - lon1)
    a = (sin(d_lat / 2) ** 2 +
         cos(radians(lat1)) * cos(radians(lat2)) *
         sin(d_lon / 2) ** 2)
    c = 2 * atan2(sqrt(a), sqrt(1 - a))
    distance = R * c
    return distance

def get_coordinates_and_distances(cities):
    """
    Uses Google Maps API to geocode city names, and then
    calculates all pairwise distances (in meters) via Haversine.
    """
    coordinates = {}
    distances = {}
    
    # Get Coordinates using Geocoding API
    for city in cities:
        geocode_result = gmaps.geocode(city)
        lat = geocode_result[0]['geometry']['location']['lat']
        lng = geocode_result[0]['geometry']['location']['lng']
        coordinates[city] = (lat, lng)
        
    # Calculate Distances using Haversine Formula
    for origin in cities:
        for destination in cities:
            if origin != destination and ((origin, destination) not in distances):
                distance = haversine(coordinates[origin][0],
                                     coordinates[origin][1],
                                     coordinates[destination][0],
                                     coordinates[destination][1])
                distances[(origin, destination)] = distance
                distances[(destination, origin)] = distance
                
    return coordinates, distances

def tsp_nearest_neighbor(coordinates, distances):
    """
    Simple nearest-neighbor approach to TSP.
    """
    unvisited = list(coordinates.keys())
    start_city = unvisited.pop(0)  # Start from the first city in the list
    path = [start_city]
    
    while unvisited:
        current_city = path[-1]
        next_city = min(unvisited,
                        key=lambda city: distances[(current_city, city)])
        path.append(next_city)
        unvisited.remove(next_city)
        
    # Return to the starting city
    # path.append(path[0])
    return path

def two_opt(path, distances):
    """
    Performs a 2-opt local search to attempt to improve the path.
    """
    improvement = True
    while improvement:
        improvement = False
        # We don't iterate through final city because path[-1] == path[0].
        for i in range(1, len(path) - 2):
            for j in range(i + 1, len(path) - 1):
                # skip adjacent indices
                if j - i == 1: 
                    continue
                old_dist = (distances[(path[i - 1], path[i])] +
                            distances[(path[j], path[j + 1])])
                new_dist = (distances[(path[i - 1], path[j])] +
                            distances[(path[i], path[j + 1])])
                if new_dist < old_dist:
                    path[i:j + 1] = reversed(path[i:j + 1])
                    improvement = True
                    break
            if improvement:
                break
    return path

def insert_knoxville(coordinates, distances, optimal_path):
    if 'Knoxville' not in coordinates:
        knoxville_coords = get_coordinates_and_distances(['Knoxville'])[0]['Knoxville']
        coordinates['Knoxville'] = knoxville_coords

    # Avoid inserting if Knoxville is already there
    if optimal_path[0] != 'Knoxville':
        optimal_path = ['Knoxville'] + optimal_path
    if optimal_path[-1] != 'Knoxville':
        optimal_path = optimal_path + ['Knoxville']

    return optimal_path

### New draw_map (Cartopy instead of Basemap)

def draw_map(ax, coordinates, path):
    ax.set_global()
    ax.add_feature(cfeature.LAND, facecolor='#2e8b57')
    ax.add_feature(cfeature.OCEAN, facecolor='#1e3f66')
    ax.add_feature(cfeature.COASTLINE)
    ax.add_feature(cfeature.BORDERS, linestyle=':')
    gl = ax.gridlines(draw_labels=False)
    gl.top_labels = gl.right_labels = False

    # Plot cities
    for city, (lat, lon) in coordinates.items():
        ax.plot(lon, lat, marker='o', markersize=6,
                markeredgecolor='white', markerfacecolor='black',
                transform=ccrs.PlateCarree(), zorder=5)

    # Use WGS84 geodesic model
    geod = Geodesic.WGS84

    for i in range(len(path) - 1):
        cityA, cityB = path[i], path[i + 1]
        latA, lonA = coordinates[cityA]
        latB, lonB = coordinates[cityB]

        line = geod.InverseLine(latA, lonA, latB, lonB)
        n_points = 30
        lats, lons = [], []

        for j in range(n_points + 1):
            s = j * line.s13 / n_points
            pos = line.Position(s, Geodesic.STANDARD | Geodesic.LONG_UNROLL)
            lats.append(pos['lat2'])
            lons.append(pos['lon2'])

        ax.plot(lons, lats,
                color='red',
                linestyle='--',
                linewidth=2,
                transform=ccrs.PlateCarree(),
                zorder=3)

### MAIN

if __name__ == "__main__":
    user_input = input("Enter a comma-separated list of cities: ")
    selected_cities = [city.strip() for city in user_input.split(",")]
    
    # Get city coords/distances
    coordinates, distances = get_coordinates_and_distances(selected_cities)
    
    # Compute a TSP path via nearest neighbor + 2-opt
    nn_path = tsp_nearest_neighbor(coordinates, distances)
    improved_path = two_opt(nn_path, distances)
    
    # Insert Knoxville into path (front and back)
    path_with_knoxville = insert_knoxville(coordinates, distances, improved_path)

    # Plot with Cartopy
    plt.figure(figsize=(19.2, 9.6))
    # Use a Mollweide projection to roughly match your old Basemap version
    ax = plt.axes(projection=ccrs.Mollweide(central_longitude=9)) # nine degrees is slightly better balanced :)
    
    draw_map(ax, coordinates, path_with_knoxville)

    plt.title("Optimal Trip Path, From and Back to Home")
    plt.show()